Let's look at the residual tendency resulting from simulation. In this cell, I load the data and compute the tendency errors.

In [ ]:
from src.data import open_data
import torch
from uwnet.thermo import compute_apparent_source

model = torch.load("../data/runs/model268-epoch5.debiased/model.pkl")

data = open_data("training")
data = data.isel(time=slice(0,100)).compute()

srcs = model.predict(data)

q2 = compute_apparent_source(data.QT, data.FQT*86400)
q1 = compute_apparent_source(data.SLI, data.FSLI*86400)

error_qt = q2 - srcs.QT
error_sli = q1 - srcs.SLI

Here is a plot of the tendency error

In [ ]:
error_qt[0,10].plot()

In [ ]:
error_sli[0,10].plot()

In [ ]:
error_qt.dropna('time').mean(['x', 'time']).plot()

In [ ]:
error_qt.dropna('time').std(['x', 'time']).plot()

The standard deviation of this residual is much larger than mean. In fact the mean should be zero over the whole dataset because this simulation is debiased.

In [ ]:
plt.hist(error_qt[:,10].values.ravel(), 100);

In [ ]:
plt.hist(error_sli[:,10].values.ravel(), 100);

# Vertical covariance of tendency error

In [ ]:
cov = (error_qt *error_qt.rename({'z': 'zp'})).mean(['time', 'x'])

In [ ]:
cov.isel(y=32).plot()

In [ ]:
cov.isel(y=10).plot()

In [ ]:
cov.isel(y=20).plot()

the covariance is more broad-based in the tropics and extra-tropics than the sub-tropics. Maybe this occurs because there is less signal there.

In any case, we can generate noise with this covariance.

# Cross covariance of Q1 and Q2 error

In [ ]:
cov = (error_qt *error_sli.rename({'z': 'zp'})).mean(['time', 'x'])

In [ ]:
cov.isel(y=32).plot(y='zp')

These errors are negative correlated in the troposhere (indicating they they might conserve MSE).

In [ ]:
cov.isel(y=10).plot(y='zp')

# Spatial Spectrum

In [ ]:
from scipy.signal import periodogram

error_qt_z10 = error_qt.isel(z=10).dropna('time')

f, pxx = periodogram(error_qt_z10.values, axis=-1)
pxx = pxx.mean(0)

plt.plot(pxx[32], label='tropics')
plt.plot(pxx[10], label='extra-tropics')
plt.legend()

This error is fairly weight in space, but does have a peak at large-scales.

# Temporal spectra

In [ ]:
from scipy.signal import periodogram

error_qt_z10 = error_qt.isel(z=10).dropna('time')

f, pxx = periodogram(error_qt_z10.values, 1/.125, axis=error_qt_z10.get_axis_num('time'))
pxx = pxx.mean(-1).T

plt.plot(f, pxx[32], label='tropics')
plt.plot(f, pxx[10], label='extra-tropics')
plt.legend()

The temporal spectra is significantly less weight. Especially in the tropics.

# Predicted vs Error

In [ ]:
plt.hexbin(
    srcs.QT[:,10, 32].values.ravel(), error_qt[:,10, 32].values.ravel())
plt.xlim([-5,5])
plt.ylim([-5,5])
plt.xlabel('QT Pred')
plt.ylabel('Q2 - QT Pred');

If anything it seems like the NN over-estimates the amount of rainfall.